In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from pickle import load

In [0]:
input_data = load(open('/content/drive/My Drive/Colab Notebooks/Movies/all_movie_scripts', 'rb'))

In [0]:
type(input_data[0])

In [0]:
input_data

In [0]:
import spacy
import re
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'net'])

In [0]:
def process_data(data):
    return [token.text.lower() for token in nlp(data) if token.text not in '!@#$%^&*()_+~`{}[]:";,./<>?\n\n\n\t\b-...."']

def form_sequences(data):
    seq_len = 50 + 1
    sequences = []
    for i in range(seq_len, len(input_data)):
        sequences.append(input_data[i-seq_len:i])
    return sequences

In [0]:
input_data = ' '.join(input_data)

In [0]:
len(input_data)

In [0]:
input_data = process_data(input_data)

In [0]:
sequences = form_sequences(input_data)

In [57]:
len(sequences[0])

51

In [0]:
!pip install keras
!pip install tensorflow

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer()

In [0]:
tokenizer.fit_on_texts(sequences)

In [0]:
sequences = tokenizer.texts_to_sequences(sequences)

In [61]:
sequences[0]

[5128,
 102,
 144,
 102,
 144,
 46,
 87,
 107,
 38,
 63,
 428,
 61,
 1,
 335,
 19,
 230,
 35,
 11,
 26,
 3,
 44,
 1173,
 127,
 21,
 706,
 55,
 19,
 5,
 6,
 397,
 8,
 32,
 7,
 641,
 17,
 19,
 2,
 3,
 45,
 104,
 1,
 24,
 3,
 45,
 15,
 1,
 86,
 61,
 1,
 13,
 18]

In [0]:
tokenizer.word_index

In [0]:
vocab_size = len(tokenizer.word_counts)

In [0]:
import numpy as np

In [0]:
sequences = np.array(sequences)

In [0]:
sequences

In [0]:
X = sequences[:,:-1]
y = sequences[:, -1]

In [0]:
from keras.utils import to_categorical

In [0]:
X

In [0]:
y = to_categorical(y, num_classes=vocab_size+1)

In [0]:
y.shape

In [0]:
seq_len = X.shape[1]

In [0]:
seq_len

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

In [0]:
def create_model(vocab_size, seq_len):
    
    model = Sequential()
    model.add(Embedding(vocab_size, seq_len, input_length=seq_len))
    model.add(LSTM(500, return_sequences=True))
    model.add(LSTM(400))
    model.add(Dense(350, activation='relu'))
    model.add(Dense(250, activation='relu'))
    
    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    model.summary()
    
    return model

In [0]:
model = create_model(vocab_size+1, seq_len)

In [0]:
 model.fit(X, y, batch_size=200, epochs=200, verbose=1)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [0]:
model.save('zombie_movie_96.h5')
model_file = drive.CreateFile({'title' : 'zombie_movie_96.h5'})    
model_file.SetContentFile('zombie_movie_96.h5')
model_file.Upload()

In [0]:
drive.CreateFile({'id': model_file.get('id')})

In [0]:
model

In [0]:
model.evaluate(X,y)

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_index = model.predict_classes(pad_encoded)[0]
        pred_word = tokenizer.index_word[pred_word_index]
        
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [0]:
sequences[267]

In [0]:
seed_text = 'Who would have thought that an accident involving lime soda would set off the apocalypse? And when the contaminated green haze blew west. Because it turned them into... Zombies. That\'s right. Zombies!But we built a big barrier to protect us from the zombie hordes.'

In [0]:
len(seed_text)

In [67]:
generate_text(model, tokenizer, seq_len, seed_text, num_gen_words=300)

'right you can still be walking round in this old world alone cause after life is over the afterlife goes you have that maverick the skybridge so what do we have me choice then it might be here um he was probably just fit never too happy but i do n t make it i talk to pay about you she no i m trying to find us maybe its things please stop i are you and your educated skull it me a favor a asses feedback almost made it out contact dumb tape what about it a cat representation of the man defense unstable sir uh muttering very fell the hive i see the rest of you and that is that i don t want to split you though hot cross the president so there s late for us if from the post that is the cell all has four continues blades whirring they are inbound to need off it would shoot try to come back about the blockade it makes you might have done to have that now the dj and you for this thing out of here this than dying i can i shoot about this do n t get it you go thank you all shit get back to save 